<a id=top></a>
# Evaluating STIS Target Acquisitions <a class="tocSkip">

<h2>Learning Goals<span class="tocSkip"></span></h2>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Import-necessary-packages" data-toc-modified-id="Import-necessary-packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import necessary packages</a></span></li></ul></li><li><span><a href="#Define-Function-for-Plotting-Acquisition-Images-and-Performing-Target-Acquisition" data-toc-modified-id="Define-Function-for-Plotting-Acquisition-Images-and-Performing-Target-Acquisition-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define Function for Plotting Acquisition Images and Performing Target Acquisition</a></span></li><li><span><a href="#Successful-Target-Acquisition" data-toc-modified-id="Successful-Target-Acquisition-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Successful Target Acquisition</a></span></li><li><span><a href="#Target-Acquisition-Failure-Cases" data-toc-modified-id="Target-Acquisition-Failure-Cases-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Target Acquisition Failure Cases</a></span><ul class="toc-item"><li><span><a href="#Guide-Stars-not-Acquired" data-toc-modified-id="Guide-Stars-not-Acquired-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Guide Stars not Acquired</a></span></li><li><span><a href="#Guide-Star-Acquisition-Delayed" data-toc-modified-id="Guide-Star-Acquisition-Delayed-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Guide Star Acquisition Delayed</a></span></li><li><span><a href="#Lamp-not-illuminated" data-toc-modified-id="Lamp-not-illuminated-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Lamp not illuminated</a></span></li><li><span><a href="#Multiple-Bright-Sources" data-toc-modified-id="Multiple-Bright-Sources-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Multiple Bright Sources</a></span></li><li><span><a href="#Saturation-of-Pixels" data-toc-modified-id="Saturation-of-Pixels-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Saturation of Pixels</a></span></li><li><span><a href="#No-Target-in-the-Acquisition-Image" data-toc-modified-id="No-Target-in-the-Acquisition-Image-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>No Target in the Acquisition Image</a></span></li></ul></li></ul></div>

## Introduction
There are two types of STIS target acquisitions: ACQ and ACQ/PEAKUP. The ACQ is used in almost all STIS observations to center the target. The ACQ/PEAKUP can be taken after the ACQ to further refine the centering and is recommended for observations using slits with widths less than 0.2”. In this notebook, we will go through the steps for ACQ observations, and explore some success and failure cases. 


For ACQ observations, the target acquisition data has three science extensions:
- Before Coarse Slew
- Before Fine Slew
- 0.2$\times$0.2 Reference Aperture with HITM lamp

The first science extension (Coarse Phase) of the acquisition raw file is an image of target in the target acquisition sub-array based on initial pointing. Then the coarse centering is performed: the software determines the position of the target with a flux weighted pointing algorithm, and calculates the slew needed to place the target at a reference point in the target acquisition sub-array; the coarse centered image is stored in the second science extension (Fine Phase). The next step is to perform fine centering: a 32 $\times$ 32 pixel image of the reference aperture is obtained and stored in the third science extension (Reference Aperture), the location of the aperture on the detector is determined, and a fine slew is performed to center the taret in the reference aperture (to place the object precisely in a slit).

For more information on ACQ target acquisition, see [Evaluating Target Acquisitions and Guiding](https://hst-docs.stsci.edu/stisdhb/chapter-5-stis-data-analysis/5-2-evaluating-target-acquisitions-and-guiding).

### Import necessary packages

**We will import the following packages:**

- `astropy.io fits` for accessing FITS files
- `astroquery.mast Observations` for finding and downloading data from the [MAST](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html) archive
- `os` for managing system paths
- `matplotlib` for plotting data
- `stistools` for quick operations on STIS Data

In [ ]:
# Import for: Reading in fits file
from astropy.io import fits

# Import for: Downloading necessary files. (Not necessary if you choose to collect data from MAST)
from astroquery.mast import Observations

# Import for: Managing system variables and paths
import os

# Import for operations on STIS Data
import stistools

# Import for: Plotting and specifying plotting parameters
import matplotlib.pyplot as plt
import matplotlib
from astropy.visualization import ZScaleInterval
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.cmap'] = 'plasma'
matplotlib.rcParams['image.interpolation'] = 'none'
matplotlib.rcParams['figure.figsize'] = (20, 7)

## Define Function for Plotting Acquisition Images and Performing Target Acquisition

We define a function to plot the three science image extensions mentioned above, and run the stistools_tastis target acquisition method to show the result and associated information of target acquisition.

In [ ]:
def acq(obs_id):
    # Search target object by obs_id
    target = Observations.query_criteria(obs_id=obs_id)
    # Get a list of files associated with that target
    acq_list = Observations.get_product_list(target)
    # Download only the _raw and _spt fits files
    Observations.download_products(acq_list, extension=['raw.fits', 'spt.fits'])
    raw = os.path.join("./mastDownload/HST", "{}".format(obs_id), "{}_raw.fits".format(obs_id))
    
    # Plot the acquisition raw images
    with fits.open(raw) as hdu:
        initial = hdu[1].data
        confirm = hdu[4].data
        lamp = hdu[7].data
    zscaler = ZScaleInterval(contrast=0.10).get_limits 
    _, initial_max = zscaler(initial)
    _, confirm_max = zscaler(confirm)
    vmax = max(initial_max, confirm_max)
    
    # Plot the initial target position in coarse phase
    TARGA1_1 = fits.getheader(raw, 1)["TARGA1"]
    TARGA2_1 = fits.getheader(raw, 1)["TARGA2"]
    TARGA1_2 = fits.getheader(raw, 4)["TARGA1"]
    TARGA2_2 = fits.getheader(raw, 4)["TARGA2"]
    centera1 = fits.getheader(raw, 0)["CENTERA1"]
    centera2 = fits.getheader(raw, 0)["CENTERA2"]
    sizaxis1 = fits.getheader(raw, 0)["sizaxis1"]
    sizaxis2 = fits.getheader(raw, 0)["sizaxis2"]
    corner1 = centera1 - sizaxis1/2 
    corner2 = centera2 - sizaxis2/2
    
    plt.subplot(1, 3, 1)
    plt.imshow(initial, vmin=0, vmax=vmax)
    plt.plot([TARGA1_1-corner1+1], [TARGA2_1-corner2+1], "x", c="white", markersize=12)
    plt.plot([TARGA1_1-corner1+1], [TARGA2_1-corner2+1], "x", c="black", markersize=10)
    plt.title("Before Coarse Slew")
    plt.subplot(1, 3, 2)
    plt.imshow(confirm, vmin=0, vmax=vmax)
    plt.plot([TARGA1_2-corner1+1], [TARGA2_2-corner2+1], "x", c="white", markersize=12)
    plt.plot([TARGA1_2-corner1+1], [TARGA2_2-corner2+1], "x", c="black", markersize=10)
    plt.title("Before Fine Slew")
    plt.subplot(1, 3, 3)
    plt.imshow(lamp, vmin=0, vmax=6000)
    plt.title(r"$0.2 \times 0.2$ Reference Aperture with HITM lamp")
    plt.tight_layout()
    
    stistools.tastis.tastis(raw)

The PDQ quality flags of the dataset are indicated in the "QUALITY" field of the raw fits file 0th header, and telemetry comments are saved in the "QUALCOM" fields of the 0th header. The same comments can also be seen on the Preview page of the dataset on the [`MAST`](https://mast.stsci.edu/search/ui/#/hst) archive. We can get those fields to confirm the potential reasons for acquisition failure:

In [ ]:
def get_telemetry(obs_id):
    raw_file = "./mastDownload/HST/{id}/{id}_raw.fits".format(id=obs_id)
    quality = fits.getval(raw_file, "QUALITY", ext=0)
    QUALCOM1 = fits.getval(raw_file, "QUALCOM1", ext=0)
    QUALCOM2 = fits.getval(raw_file, "QUALCOM2", ext=0)
    QUALCOM3 = fits.getval(raw_file, "QUALCOM3", ext=0)
    print(f'The quality of this exposure is {quality}. {QUALCOM1} {QUALCOM2} {QUALCOM3}')

## Successful Target Acquisition
Here we present a successful target acquisition case in which the target is correctly selected from the initial pointing image, coarse centering and fine centering are successfully performed, and the target is eventually centered in the slit. The target location, coarse slew, and fine slew data are printed by stistools.tastis. Note here that the coordinate system of tastis is 1-indexed while Python is 0-indexed.

In [ ]:
acq("octka2daq")

## Target Acquisition Failure Cases
The tastis analyzes STIS target acquisitions, and prints warning information if the acquisition fails. The possible tastis failures are: 
 - Telemetry indicates that the intended exposures may not have been performed.  Check the images for signal.
 - The fine slew (to center the target in the reference aperture) is larger than 4 pixels.  This may indicate a problem with your acquisition.
 - The fluxes in the maximum checkbox in the fine and coarse stages differ by more than 25%.  This may indicate a problem with your acquisition.
 - The flux in the third image of the ACQ is lower than the typical value for the lamp; the image should be checked to see if the lamp was illuminated.
 - Saturation of pixels in the second image may have affected the final centering.
 
Sometimes a failure case might be caused by multiple acquisition warnings. In this section, we will go through several typical cases of acquisition failure. 

### Guide Stars not Acquired
The Take Data Flag (TDF) is an onboard pointing control system flag which, when ON, indicates that the spacecraft is ready for an observation and that the guide star tracking performance is nomial. If the TDF is down, exposures are not performed until TDF comes up.

If the guide stars are not acquired during acquisition, the TDF will be turned down and no exposures will be performed. Therefore all acquisition images will be dark.

In this failure case, as telemetry indicates, the TDF is down at the start and no exposure was performed. Thus there is not a clear bright source in the Before Coarse Slew and Before Fine Slew images. Usually, exposure failures can also cause other issues such as fine slew larger than 4 pixels or Ratio of Flux in Max Checkbox in Fine & Coarse Stages Greater than 25%, because the tastis algorithm cannot pick up the proper bright source in the image to perform the following steps of acquisition.

In [ ]:
acq("oeds03rzq")

Telemetry of this observation confirms that the guide star acquisition failed and the aperture door was closed throughout the exposure:

In [ ]:
get_telemetry("oeds03rzq")

### Guide Star Acquisition Delayed
In this failure case, the guide star acquisition is delayed during the coarse phase, but the guide stars are then acquired in the fine phase. Therefore the first acquisition image is dark while the target is in the second acquisition image. Eventually the target will still be centered, but the large fine slews may result in less precise positioning.

In [ ]:
acq("oe1l55vjq")

Telemetry of this observation suggests that the guide star acquisition was delayed and the exposure was shortened:

In [ ]:
get_telemetry("oe1l55vjq")

### Lamp not illuminated
As shown in the reference aperture, the reference aperture HITM lamp is dark and it has a flux smaller than the typical minumun value for the lamp. Usually, this kind of failure occurs if the TDF is down when the lamp image is taken; however, in some rare cases, the lamp might not be functioning. If the lamp image is blank, the target might not be centered in the reference aperture.

In [ ]:
acq("oec62hj6q")

The LOCKLOST quality flag suggests that guide star was lost during the guide period, and thus the exposure was shortened and the lamp was not illuminated:

In [ ]:
get_telemetry("oec62hj6q")

### Multiple Bright Sources
If there are multiple bright sources in a single acquisition image, the maximum checkbox will go through the image and find the brightest one. However, the presence of a source brighter than the target might confuse the algorithm and the acquisition might center at the wrong target.

In [ ]:
acq("odkt01bvq")

### Saturation of Pixels
For the CCD detector, if a pixel is saturated, the excess charge will leak to adjacent pixels in the column. Saturations of pixels will affect the target flux in the max checkbox and the final centering.

In [ ]:
acq("oe4h03rdq")

If we zoom in to the center of the second acquisition image and change the color scale, we can see that the pixels in the center are saturated, and the excess charges follow downward along the column so that the pixels below are affected.

In [ ]:
raw = os.path.join("./mastDownload/HST", "{}".format("oe4h03rdq"), "{}_raw.fits".format("oe4h03rdq"))
with fits.open(raw) as hdu:
    confirm = hdu[4].data
plt.imshow(confirm, vmin=0, vmax=1e4)
plt.xlim(32, 62)
plt.ylim(35, 65)

### No Target in the Acquisition Image
In this failure case, the guide stars were acquired successfully, and the TDF is always on through the exposure. However, the first and second acquisition images are blank and no target is acquired. This kind of failure happens if the submitted target location/orientation is wrong, or the source is too faint to be observed.

In [ ]:
acq("ocyg39yoq")


---
## About this Notebook <a class="tocSkip">
**Author:** [Keyi Ding](kding@stsci.edu)

**Updated On:** 2023-04-12


> *This tutorial was generated to be in compliance with the [STScI style guides](https://github.com/spacetelescope/style-guides) and would like to cite the [Jupyter guide](https://github.com/spacetelescope/style-guides/blob/master/templates/example_notebook.ipynb) in particular.*
## Citations <a class="tocSkip">

If you use `astropy`, `matplotlib`, `astroquery`, or `numpy` for published research, please cite the
authors. Follow these links for more information about citations:

* [Citing `matplotlib`](https://matplotlib.org/stable/users/project/citing.html)
* [Citing `numpy`](https://numpy.org/citing-numpy/)
* [Citing `astroquery`](https://astroquery.readthedocs.io/en/latest/)
* [Citing `astropy`](https://www.astropy.org/acknowledging.html)

<hr>

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 